In [1]:
import tensorflow as tf
slim = tf.contrib.slim
import numpy as np

C:\Users\home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = tf.zeros(shape=[16,28,28,3])
z_input = tf.random_normal([16, 100], mean=0, stddev=1)

In [3]:
def count_parameters(network_variables, name):
    """
    This method counts the tota
    l number of unique parameters for a list of variable objects
    :param network_variables: A list of tf network variable objects
    :param name: Name of the network
    """
    total_parameters = 0
    for variable in network_variables:
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value

        total_parameters += variable_parametes
    print(name, total_parameters)

In [ ]:
tf.reset_default_graph()

In [4]:
class UResNetGenerator:
    def __init__(self, layer_sizes, layer_padding, batch_size, num_channels=1,
                 inner_layers=0, name="g"):
        """
        Initialize a UResNet generator.
        :param layer_sizes: A list with the filter sizes for each MultiLayer e.g. [64, 64, 128, 128]
        :param layer_padding: A list with the padding type for each layer e.g. ["SAME", "SAME", "SAME", "SAME"]
        :param batch_size: An integer indicating the batch size
        :param num_channels: An integer indicating the number of input channels
        :param inner_layers: An integer indicating the number of inner layers per MultiLayer
        """
        self.reuse = False
        self.batch_size = batch_size
        self.num_channels = num_channels
        self.layer_sizes = layer_sizes
        self.layer_padding = layer_padding
        self.inner_layers = inner_layers
        self.conv_layer_num = 0
        self.build = True
        self.name = name
        self.encoder_layers = []
        self.decoder_layers = []
        self.num_filters = 8
                
    def upscale(self, x, h_size, w_size):
        """
        Upscales an image using nearest neighbour
        :param x: Input image
        :param h_size: Image height size
        :param w_size: Image width size
        :return: Upscaled image
        """
        [b, h, w, c] = [int(dim) for dim in x.get_shape()]

        return tf.image.resize_nearest_neighbor(x, (h_size, w_size))
    
    def encoder_block(self, x1, num, scope = None): 
        with tf.variable_scope(scope +'/encoder_block'):
            with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                                num_outputs = 64, padding = 'SAME',
                                kernel_size = [3,3], stride = (1,1),
                                activation_fn = tf.nn.leaky_relu,
                                normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params):
                
                conv1_1 = slim.conv2d(x1)
                output1_1 = tf.concat([conv1_1, x1], axis=3)
                print(output1_1)

                conv1_2 = slim.conv2d(output1_1)
                output1_2 = tf.concat([conv1_2, output1_1], axis=3)
                print(output1_2)

                conv1_3 = slim.conv2d(output1_2)
                output1_3 = tf.concat([conv1_3, output1_2], axis=3)
                print(output1_3)

                conv1_4 = slim.conv2d(output1_3, stride=(2,2))
                output = slim.dropout(conv1_4, keep_prob=0.5)
                self.encoder_layers.append(output)
                print('before inprejec : ', output)
                
                if num == 3:
                    pass
                else :
                    input_projection = slim.conv2d(conv1_3, num_outputs=conv1_3.get_shape()[3], stride=(2,2),
                                                   activation_fn= None, normalizer_fn= None)
                    output = tf.concat([output, input_projection], axis=3)
                print(output)

        return output
    
    def decoder_block(self, x1): 
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                            num_outputs = 64, padding = 'SAME',
                            kernel_size = [3,3], stride = (1,1),
                            activation_fn = tf.nn.leaky_relu,
                            normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params):

            conv1_1 = slim.conv2d(x1)
            output1_1 = tf.concat([conv1_1, x1], axis=3)
            print(output1_1)

            conv1_2 = slim.conv2d(output1_1)
            output1_2 = tf.concat([conv1_2, output1_1], axis=3)
            print(output1_2)

            conv1_3 = slim.conv2d(output1_2)
            output1_3 = tf.concat([conv1_3, output1_2], axis=3)
            print(output1_3)

            #output1_3 = self.upscale(output1_3, h, h)
            conv1_4 = slim.conv2d_transpose(output1_3, stride=(2,2))
            self.decoder_layers.append(conv1_4)
            output = slim.dropout(conv1_4, keep_prob=0.5)

            input_projection = slim.conv2d_transpose(conv1_3, num_outputs=conv1_3.get_shape()[3], stride=(2,2),
                                           activation_fn= None, normalizer_fn= None)
            output = tf.concat([output, input_projection], axis=3)
            print(output)

        return output
    
    def z_noise_concat(self, inputs, z_inputs, h, w):
        z_dense = tf.layers.dense(z_inputs, h*w*self.num_filters)
        z_noise = tf.reshape(z_dense, [self.batch_size, h, w, self.num_filters])
        z_noise_concat = tf.concat([inputs, z_noise], axis= 3)
        
        self.num_filters = np.int(self.num_filters / 2)
        print(self.num_filters)
        
        return z_noise_concat
    
        
    def __call__(self, z_inputs, conditional_input, training=False, dropout_rate=0.0):
        """
        Apply network on data.
        :param z_inputs: Random noise to inject [batch_size, z_dim]
        :param conditional_input: A batch of images to use as conditionals [batch_size, height, width, channels]
        :param training: Training placeholder or boolean
        :param dropout_rate: Dropout rate placeholder or float
        :return: Returns x_g (generated images), encoder_layers(encoder features), decoder_layers(decoder features)
        """
        self.batch_norm_params = {'decay' : 0.99, 'scale' : True, 'center' : True,
                                 'is_training' : training, 'renorm' : True}

        with tf.variable_scope(self.name, reuse=self.reuse):
            # reshape from inputs
            conditional_input = tf.convert_to_tensor(conditional_input)
            
            with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                       num_outputs = 64, padding = 'SAME',
                        kernel_size = [3,3], stride = (1,1), activation_fn = None):
                with tf.variable_scope(self.name + 'first_en_conv'):
                    conv = slim.conv2d(conditional_input, stride=(2,2))
                    self.encoder_layers.append(conv)
                    
                    input_projection = slim.conv2d(conditional_input, num_outputs=conditional_input.get_shape()[3], stride=(2,2))
                    output1 = tf.concat([conv, input_projection], axis=3)

                en1 = self.encoder_block(output1, 1, scope='first_en_block') #[B,7, 7, 64]
                en2 = self.encoder_block(en1, 2, scope='second_en_block') #[B,4,4,64]
                en3 = self.encoder_block(en2, 3, scope='third_en_block') #[b,2,2,64]
                #end encoder
                print('end encoder')
                print('encoder_layers[0] ', self.encoder_layers[0])
                
                with tf.variable_scope(self.name + '/First_de_conv'):
                    self.decoder_layers.append(en3)
                    input_noise = self.z_noise_concat(en3, z_inputs, 2, 2)                #[b,2,2,72]
                    
                with tf.variable_scope(self.name + '/First_de_block'):
                    print('input : ' ,input_noise)
                    de_conv1 = self.decoder_block(input_noise)                            #[b, 4, 4, 64]
                    de_conv1 = tf.concat([de_conv1, self.encoder_layers[2]], axis=3)
                    de_conv1_noise = self.z_noise_concat(de_conv1, z_inputs, 4, 4)
                    
                with tf.variable_scope(self.name + '/Second_de_block'):
                    de_conv2 = self.decoder_block(de_conv1_noise)                         #[b, 8, 8, 64]
                    de_conv2_noise = self.z_noise_concat(de_conv2, z_inputs, 8, 8)
                    de_conv2 = self.upscale(de_conv2_noise, 7, 7)
                    de_conv2 = tf.concat([de_conv2, self.encoder_layers[1]], axis=3)
                    
                with tf.variable_scope(self.name + '/Third_de_block'):
                    de_conv3 = self.decoder_block(de_conv2)                               #[b, 14, 14 ,64]
                    de_conv3 = tf.concat([de_conv3, self.encoder_layers[0]], axis=3)
                    
                with tf.variable_scope(self.name + '/Forth_de_block'):
                    de_conv4 = self.decoder_block(de_conv3)                               #[b, 28, 28 ,64]
                    de_conv4 = tf.concat([de_conv4, conditional_input], axis=3)
                    
                with tf.variable_scope(self.name + '/Last_de_block'):
                    de_conv5_1 = slim.conv2d(de_conv4)
                    de_conv5_1 = tf.concat([de_conv5_1, de_conv4], axis=3)
                    
                    de_conv5_2 = slim.conv2d(de_conv5_1)
                    de_conv5_2 = tf.concat([de_conv5_2, de_conv5_1], axis=3)
                    
                with tf.variable_scope(self.name + '/P_process'):
                    de_conv = slim.conv2d(de_conv5_2)
                    de_conv = slim.conv2d(de_conv)
                
                with tf.variable_scope('g_tanh'):
                    gan_decoder = tf.tanh(de_conv, name='outputs')
                
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)

        if self.build:
            count_parameters(self.variables, 'generator_parameter_num')
        self.build = False
        
        return gan_decoder, self.encoder_layers, self.decoder_layers        
                          

In [5]:
a = UResNetGenerator([64,64,64,64], ['SAME','SAME','SAME','SAME'], 16)

In [6]:
a(z_inputs=z_input , conditional_input= x)

Tensor("g/first_en_block/encoder_block/concat:0", shape=(16, 14, 14, 131), dtype=float32)
Tensor("g/first_en_block/encoder_block/concat_1:0", shape=(16, 14, 14, 195), dtype=float32)
Tensor("g/first_en_block/encoder_block/concat_2:0", shape=(16, 14, 14, 259), dtype=float32)
before inprejec :  Tensor("g/first_en_block/encoder_block/Dropout/dropout_1/mul:0", shape=(16, 7, 7, 64), dtype=float32)
Tensor("g/first_en_block/encoder_block/concat_3:0", shape=(16, 7, 7, 128), dtype=float32)
Tensor("g/second_en_block/encoder_block/concat:0", shape=(16, 7, 7, 192), dtype=float32)
Tensor("g/second_en_block/encoder_block/concat_1:0", shape=(16, 7, 7, 256), dtype=float32)
Tensor("g/second_en_block/encoder_block/concat_2:0", shape=(16, 7, 7, 320), dtype=float32)
before inprejec :  Tensor("g/second_en_block/encoder_block/Dropout/dropout_1/mul:0", shape=(16, 4, 4, 64), dtype=float32)
Tensor("g/second_en_block/encoder_block/concat_3:0", shape=(16, 4, 4, 128), dtype=float32)
Tensor("g/third_en_block/encode

(<tf.Tensor 'g/g_tanh/outputs:0' shape=(16, 28, 28, 64) dtype=float32>,
 [<tf.Tensor 'g/gfirst_en_conv/Conv/BiasAdd:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'g/first_en_block/encoder_block/Dropout/dropout_1/mul:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'g/second_en_block/encoder_block/Dropout/dropout_1/mul:0' shape=(16, 4, 4, 64) dtype=float32>,
  <tf.Tensor 'g/third_en_block/encoder_block/Dropout/dropout_1/mul:0' shape=(16, 2, 2, 64) dtype=float32>],
 [<tf.Tensor 'g/third_en_block/encoder_block/Dropout/dropout_1/mul:0' shape=(16, 2, 2, 64) dtype=float32>,
  <tf.Tensor 'g/g/First_de_block/Conv2d_transpose/LeakyRelu:0' shape=(16, 4, 4, 64) dtype=float32>,
  <tf.Tensor 'g/g/Second_de_block/Conv2d_transpose/LeakyRelu:0' shape=(16, 8, 8, 64) dtype=float32>,
  <tf.Tensor 'g/g/Third_de_block/Conv2d_transpose/LeakyRelu:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'g/g/Forth_de_block/Conv2d_transpose/LeakyRelu:0' shape=(16, 28, 28, 64) dtype=float32>])